# Pandas 3

---

## Content

- Introduction to IMDB use case
    - Merging `movies` & `directors` datasets
    - IMDB data exploration (Post-read)
- `apply()`
- `groupby()`
    - Group based Aggregation
    - Group based Filtering
    - Group based Apply

---

## **IMDB Movies Data**

- Imagine you are working as a Data Scientist for an analytics firm.
- Your task is to analyse some **movie trends** for a client.
- **IMDB** has an online database of information related to movies.

Let's download and read the IMDB dataset -

- File1: https://drive.google.com/file/d/1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd/view?usp=sharing
- File2: https://drive.google.com/file/d/1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm/view?usp=sharing


In [ ]:
!pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 5.1.0
    Uninstalling gdown-5.1.0:
      Successfully uninstalled gdown-5.1.0


In [1]:
!gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /content/movies.csv
100% 112k/112k [00:00<00:00, 27.5MB/s]


In [2]:
!gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm

Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /content/directors.csv
100% 65.4k/65.4k [00:00<00:00, 47.3MB/s]


Here we have two CSV files -
- `movies.csv`
- `directors.csv`

In [4]:
import pandas as pd
import numpy as np

In [5]:
movies = pd.read_csv('movies.csv')
movies.head()

,Unnamed: 0,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday
1,1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday
2,2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday
3,3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday
4,5,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday


**So what kind of questions can we ask from this dataset?**

- **Top 10 most popular movies**, using `popularity`.
- Find the **highest rated movies**, using `vote_average`.
- We can find number of **movies released per year**.
- Find **highest budget movies in a year** using both `budget` and `year`.

**But can we ask more interesting/deeper questions?**

- Do you think we can find the **most productive directors**?
- Which **directors produce high budget films**?
- **Highest and lowest rated movies for every month** in a particular year?

Notice that there's a column **Unnamed: 0** which represents nothing but the index of a row.

**How to get rid of this `Unnamed: 0` col?**

In [6]:
movies = pd.read_csv('movies.csv', index_col=0)
movies.head()

,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday
2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday
5,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday


`index_col=0` explicitly states to treat the first column as the index.

The default value is `index_col=None`

In [7]:
movies.shape

(1465, 11)

The `movies` dataframe contains 1465 rows and 11 columns.

In [8]:
directors = pd.read_csv('directors.csv', index_col=0)
directors.head()

,director_name,id,gender
0,James Cameron,4762,Male
1,Gore Verbinski,4763,Male
2,Sam Mendes,4764,Male
3,Christopher Nolan,4765,Male
4,Andrew Stanton,4766,Male


In [9]:
directors.shape

(2349, 3)

The `directors` dataframe contains 2349 rows and 3 columns.

---

### Merging `movies` & `directors` datasets

**How can we know the details about the Director of a particular movie?**

- We will have to merge these two datasets.

**So on which column we should merge?**

We will use the **ID** columns (representing unique directors) in both the datasets.

If you observe,

- `director_id` of movies are taken from `id` of directors.
- Thus, we can merge our dataframes based on these two columns as **keys**.

Before that, let's first check the number of unique directors in our `movies` dataset.

**How do we get the number of unique directors in `movies`?**


In [ ]:
movies['director_id'].nunique()

199

Recall, we had learnt about `nunique()` earlier.

In [ ]:
directors['id'].nunique()

2349

**Summary:**

- `movies` dataset: 1465 rows, but only 199 unique directors
- `directors` dataset: 2349 unique directors (equal to the no. of rows)

**What can we infer from this?**

- The directors in `movies` data is a subset of directors in `directors` data.

**How can we check if all `director_id` values are present in `id`?**

In [10]:
movies['director_id'].isin(directors['id'])

,director_id
0,True
1,True
2,True
3,True
5,True
...,...
4736,True
4743,True
4748,True
4749,True


The `isin()` method checks if a column contains the specified value(s).

**How is `isin` different from Python's `in`?**

- `in` works for **one element** at a time.
- `isin` does this for **all the values** in the column.

If you notice,

- This is like a **boolean mask**.
- It returns a dataframe similar to the original one.
- For rows with values of `director_id` present in `id`, it returns True, else False.

**How can we check if there's any False here?**

In [ ]:
np.all(movies['director_id'].isin(directors['id']))

True

Let's finally merge the two dataframes.

Do we need to keep **all the rows for movies**? Yes!

Do we need to keep **all the rows of directors**? No.

- Only the ones for which we have a corresponding row in `movies`.

**So which `join` type do you think we should apply here?**

- `LEFT` Join

In [11]:
data = movies.merge(directors, how='left', left_on='director_id',right_on='id')
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day,director_name,id_y,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday,James Cameron,4762,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday,Gore Verbinski,4763,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday,Sam Mendes,4764,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday,Christopher Nolan,4765,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday,Sam Raimi,4767,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,4809,1978,May,Monday,Martin Scorsese,4809,Male
1461,48370,27000,19,3151130,Clerks,7.4,755,5369,1994,Sep,Tuesday,Kevin Smith,5369,Male
1462,48375,0,7,0,Rampage,6.0,131,5148,2009,Aug,Friday,Uwe Boll,5148,Male
1463,48376,0,3,0,Slacker,6.4,77,5535,1990,Jul,Friday,Richard Linklater,5535,Male


Notice the two strange id columns - `id_x` and `id_y`.

**What do you think these newly created columns are?**

Since the columns with name `id` are present in both the dataframes,
- `id_x` represents **id values from movie df**
- `id_y` represents **id values from directors df**

**Do you think any column is redundant here and can be dropped?**

- `id_y` is redundant as it is the same as `director_id`
- But we don't require the `director_id` any further.

So we can simply drop these features -

In [12]:
data.drop(['director_id','id_y'], axis=1, inplace=True)
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male


In [13]:
# info about table
# Give info of all the columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_x           1465 non-null   int64  
 1   budget         1465 non-null   int64  
 2   popularity     1465 non-null   int64  
 3   revenue        1465 non-null   int64  
 4   title          1465 non-null   object 
 5   vote_average   1465 non-null   float64
 6   vote_count     1465 non-null   int64  
 7   year           1465 non-null   int64  
 8   month          1465 non-null   object 
 9   day            1465 non-null   object 
 10  director_name  1465 non-null   object 
 11  gender         1341 non-null   object 
dtypes: float64(1), int64(6), object(5)
memory usage: 137.5+ KB


In [14]:
# describe
data.describe()
# gives numerical statistics of all the columns

,id_x,budget,popularity,revenue,vote_average,vote_count,year
count,1465.000000,1.465000e+03,1465.000000,1.465000e+03,1465.000000,1465.000000,1465.000000
mean,45225.191126,4.802295e+07,30.855973,1.432539e+08,6.368191,1146.396587,2002.615017
std,1189.096396,4.935541e+07,34.845214,2.064918e+08,0.818033,1578.077438,8.680141
min,43597.000000,0.000000e+00,0.000000,0.000000e+00,3.000000,1.000000,1976.000000
25%,44236.000000,1.400000e+07,11.000000,1.738013e+07,5.900000,216.000000,1998.000000
50%,45022.000000,3.300000e+07,23.000000,7.578164e+07,6.400000,571.000000,2004.000000
75%,45990.000000,6.600000e+07,41.000000,1.792469e+08,6.900000,1387.000000,2009.000000
max,48395.000000,3.800000e+08,724.000000,2.787965e+09,8.300000,13752.000000,2016.000000


In [16]:
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.0,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300.0,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245.0,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250.0,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258.0,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male


In [15]:
# budget and revenue are large numbers
# converting them to millions
data["revenue"] = (data["revenue"] / 1000000).round(2)
data["budget"] = (data["budget"] / 1000000).round(2)

In [20]:
# Print All the movies which has vote avg > 7
df2 = data.loc[data["vote_average"] > 7]
df2

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
14,43616,250.00,120,956.02,The Hobbit: The Battle of the Five Armies,7.1,4760,2014,Dec,Wednesday,Peter Jackson,Male
16,43619,250.00,94,958.40,The Hobbit: The Desolation of Smaug,7.6,4524,2013,Dec,Wednesday,Peter Jackson,Male
19,43622,200.00,100,1845.03,Titanic,7.5,7562,1997,Nov,Tuesday,James Cameron,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,48321,0.01,20,7.00,Eraserhead,7.5,485,1977,Mar,Saturday,David Lynch,Male
1457,48323,0.00,5,0.00,The Mighty,7.1,51,1998,Oct,Friday,Peter Chelsom,Male
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male


---

In [ ]:
# How to filter those records where movies released in the year(2015,2016,2012) from the above dataset ?
data[data["year"].isin([2015, 2016,2017])]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
5,43606,250000000,155,873260194,Batman v Superman: Dawn of Justice,5.7,7004,2016,Mar,Wednesday,Zack Snyder,Male
30,43641,190000000,102,1506249360,Furious 7,7.3,4176,2015,Apr,Wednesday,James Wan,Male
38,43653,185000000,65,343471816,Star Trek Beyond,6.6,2568,2016,Jul,Thursday,Justin Lin,NaN
42,43658,176000003,85,183987723,Jupiter Ascending,5.2,2768,2015,Feb,Wednesday,Lilly Wachowski,Female
...,...,...,...,...,...,...,...,...,...,...,...,...
1370,47343,4000000,24,52425855,The Boy Next Door,4.1,1022,2015,Jan,Friday,Rob Cohen,Male
1395,47575,3000000,3,0,Amnesiac,4.1,52,2015,Aug,Friday,Michael Polish,Male
1410,47719,13500000,5,187674,Stonewall,5.2,32,2015,Sep,Friday,Roland Emmerich,Male
1439,48146,0,10,316472,Good Kill,5.9,208,2015,Apr,Thursday,Andrew Niccol,Male


In [22]:
# Display the title and director-name of the movies with vote_avg > 7
df2.loc[:, ["title", "director_name"]]

,title,director_name
0,Avatar,James Cameron
3,The Dark Knight Rises,Christopher Nolan
14,The Hobbit: The Battle of the Five Armies,Peter Jackson
16,The Hobbit: The Desolation of Smaug,Peter Jackson
19,Titanic,James Cameron
...,...,...
1456,Eraserhead,David Lynch
1457,The Mighty,Peter Chelsom
1458,Pi,Darren Aronofsky
1460,The Last Waltz,Martin Scorsese


In [24]:
# Combining both - returns title and director_name of the movies where vote_average > 7
# here we applied filtering on rows ( vote_average > 7) and filtering on columns ( selecting title and director_name )
data.loc[data["vote_average"] > 7, ["title", "director_name"]]

,title,director_name
0,Avatar,James Cameron
3,The Dark Knight Rises,Christopher Nolan
14,The Hobbit: The Battle of the Five Armies,Peter Jackson
16,The Hobbit: The Desolation of Smaug,Peter Jackson
19,Titanic,James Cameron
...,...,...
1456,Eraserhead,David Lynch
1457,The Mighty,Peter Chelsom
1458,Pi,Darren Aronofsky
1460,The Last Waltz,Martin Scorsese


In [ ]:
# Display only those movies where the vote_avg > 7 and they have been released in or after 2015
# when writing multiple conditions in masking keep brackets for each condition seperately
# and use & for and | for or
data.loc[data['vote_average'] > 7 and data['year'] >= 2015]

In [25]:
data.loc[(data['vote_average'] > 7) & (data['year'] >= 2015)]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
30,43641,190.0,102,1506.25,Furious 7,7.3,4176,2015,Apr,Wednesday,James Wan,Male
78,43724,150.0,434,378.86,Mad Max: Fury Road,7.2,9427,2015,May,Wednesday,George Miller,Male
106,43773,135.0,100,532.95,The Revenant,7.3,6396,2015,Dec,Friday,Alejandro González Iñárritu,Male
162,43867,108.0,167,630.16,The Martian,7.6,7268,2015,Sep,Wednesday,Ridley Scott,Male
312,44128,75.0,48,108.15,The Man from U.N.C.L.E.,7.1,2265,2015,Aug,Thursday,Guy Ritchie,Male
394,44281,44.0,68,155.76,The Hateful Eight,7.6,4274,2015,Dec,Friday,Quentin Tarantino,Male
625,44770,35.0,53,194.56,The Intern,7.1,1881,2015,Sep,Thursday,Nancy Meyers,Female
635,44784,40.0,48,165.48,Bridge of Spies,7.2,2583,2015,Oct,Thursday,Steven Spielberg,Male
808,45194,30.0,65,91.71,Southpaw,7.3,2067,2015,Jun,Monday,Antoine Fuqua,Male
833,45293,28.0,61,201.63,Straight Outta Compton,7.7,1355,2015,Aug,Thursday,F. Gary Gray,Male


In [26]:
# Display those rows where movie has been released on Fri and Sat
data.loc[data["day"].isin(["Friday", "Saturday"])]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
12,43614,380.00,135,1045.71,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,Male
22,43627,200.00,35,783.77,Spider-Man 2,6.7,4321,2004,Jun,Friday,Sam Raimi,Male
25,43632,150.00,21,836.30,Transformers: Revenge of the Fallen,6.0,3138,2009,Jun,Friday,Michael Bay,Male
40,43656,200.00,45,769.65,2012,5.6,4903,2009,Oct,Saturday,Roland Emmerich,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1457,48323,0.00,5,0.00,The Mighty,7.1,51,1998,Oct,Friday,Peter Chelsom,Male
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,Male
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,Male


In [27]:
data.loc[(data["day"] == "Friday") | (data["day"] ==  "Saturday")]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
12,43614,380.00,135,1045.71,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,Male
22,43627,200.00,35,783.77,Spider-Man 2,6.7,4321,2004,Jun,Friday,Sam Raimi,Male
25,43632,150.00,21,836.30,Transformers: Revenge of the Fallen,6.0,3138,2009,Jun,Friday,Michael Bay,Male
40,43656,200.00,45,769.65,2012,5.6,4903,2009,Oct,Saturday,Roland Emmerich,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1457,48323,0.00,5,0.00,The Mighty,7.1,51,1998,Oct,Friday,Peter Chelsom,Male
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,Male
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,Male


### Post-read

- [IMDB data exploration](https://colab.research.google.com/drive/1yrfHSQYUMxxLKGUG-gCPf-R232BuimiR?usp=sharing)

From here, we have the opportunity to delve into various aspects of the data, such as:

- Converting the revenue values into Millions of USD.
- Identifying the Top 5 most popular movies.

... and so on.

This task is for you to explore the data on your own.

Additionally, we've provided a notebook (accessible as **post-lecture content**) where this analysis has been conducted.

---

### `apply()`

- It is used apply a function along an axis of the DataFrame/Series.

Say we want to convert the data in `Gender` column into numerical format.

Basically,
- 0 for Male
- 1 for Female

**How can we encode the values in the `Gender` column?**

Let's first write a function to do it for a single value.

In [28]:
def encode(data):
  if data == "Male":
    return 0
  else:
    return 1

**Now how can we apply this function to the whole column?**

In [29]:
data['gender'] = data['gender'].apply(encode)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0
1461,48370,0.03,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0


Notice how this is similar to using `Vectorization` in Numpy.

**How to apply a function on multiple columns?**

Let's say we want to find the sum of `revenue` and `budget` per movie?

In [30]:
data[['revenue', 'budget']].apply(np.sum) # by default axis = 0 ( Vertical )

,0
revenue,209867.04
budget,70353.62


We can pass multiple columns by packing them within `[]`.

But there's a mistake here. We wanted our results per movie (i.e. per row)

But we're getting the sum of the columns.

In [31]:
data[['revenue', 'budget']].apply(np.sum, axis=1)

,0
0,3024.97
1,1261.00
2,1125.67
3,1334.94
4,1148.87
...,...
1460,0.32
1461,3.18
1462,0.00
1463,0.00


By setting the `axis=1`, every row of `revenue` was added to same row of `budget`.

**What does this `axis` mean in apply?**
- `axis=0` $\rightarrow$ It will apply to **each column**
- `axis=1` $\rightarrow$ It will apply to **each row**
  
Note that **by default, axis=0**.

**Similarly, how can I find the `profit` per movie (revenue-budget)?**

In [32]:
# We define a function to calculate profit

def profit(x):
  return x['revenue']-x['budget']
data['profit'] = data[['revenue', 'budget']].apply(profit, axis = 1)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550.97
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661.00
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635.67
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834.94
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,0.32
1461,48370,0.03,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3.12
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0.00
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0.00


---

### What is Grouping?

In simple terms, we could understood it through - Split, Apply, Combine

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781491912126/files/assets/pyds_03in01.png" height="350" width="700"/>

1. **Split**: Breaking up and grouping a DataFrame depending on the value of the specified key.

2. **Apply**: Computing some function, usually an aggregate, transformation, or filtering, within the individual groups.

3. **Combine**: Merging the results of these operations into an output array.

---

### Group based Aggregation

We use different aggregate functions like `mean`, `sum`, `min`, `max`, `count` etc. on columns while grouping.

Let's group our data director-wise.

In [33]:
data.groupby('director_name')

Notice,
- It's a **DataFrameGroupBy** type object
- **NOT a DataFrame** type object

**What's the number of groups our data is divided into?**

In [34]:
data.groupby('director_name').ngroups

199

Based on this grouping, we can find which keys belong to which group.

In [ ]:
data.groupby('director_name').groups

# value for each key is list of indices

{'Adam McKay': [176, 323, 366, 505, 839, 916], 'Adam Shankman': [265, 300, 350, 404, 458, 843, 999, 1231], 'Alejandro González Iñárritu': [106, 749, 1015, 1034, 1077, 1405], 'Alex Proyas': [95, 159, 514, 671, 873], 'Alexander Payne': [793, 1006, 1101, 1211, 1281], 'Andrew Adamson': [11, 43, 328, 501, 947], 'Andrew Niccol': [533, 603, 701, 722, 1439], 'Andrzej Bartkowiak': [349, 549, 754, 911, 924], 'Andy Fickman': [517, 681, 909, 926, 973, 1023], 'Andy Tennant': [314, 320, 464, 593, 676, 885], 'Ang Lee': [99, 134, 748, 840, 1089, 1110, 1132, 1184], 'Anne Fletcher': [610, 650, 736, 789, 1206], 'Antoine Fuqua': [310, 338, 424, 467, 576, 808, 818, 1105], 'Atom Egoyan': [946, 1128, 1164, 1194, 1347, 1416], 'Barry Levinson': [313, 319, 471, 594, 878, 898, 1013, 1037, 1082, 1143, 1185, 1345, 1378], 'Barry Sonnenfeld': [13, 48, 90, 205, 591, 778, 783], 'Ben Stiller': [209, 212, 547, 562, 850], 'Bill Condon': [102, 307, 902, 1233, 1381], 'Bobby Farrelly': [352, 356, 481, 498, 624, 630, 654, 80

**What if we want to extract data of a particular group from this list?**

In [35]:
data.groupby('director_name').get_group('Alexander Payne')

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
793,45163,30.0,19,105.83,About Schmidt,6.7,362,2002,Dec,Friday,Alexander Payne,1,75.83
1006,45699,20.0,40,177.24,The Descendants,6.7,934,2011,Sep,Friday,Alexander Payne,1,157.24
1101,46004,16.0,23,109.50,Sideways,6.9,478,2004,Oct,Friday,Alexander Payne,1,93.50
1211,46446,12.0,29,17.65,Nebraska,7.4,636,2013,Sep,Saturday,Alexander Payne,1,5.65
1281,46813,0.0,13,0.00,Election,6.7,270,1999,Apr,Friday,Alexander Payne,1,0.00


**How can we find the count of movies by each director?**

In [ ]:
data.groupby('director_name')['title'].count()

director_name
Adam McKay                      6
Adam Shankman                   8
Alejandro González Iñárritu     6
Alex Proyas                     5
Alexander Payne                 5
                               ..
Wes Craven                     10
Wolfgang Petersen               7
Woody Allen                    18
Zack Snyder                     7
Zhang Yimou                     6
Name: title, Length: 199, dtype: int64

**How to find multiple aggregates of any feature?**

Finding the very first year and the latest year a director released a movie i.e basically the **min** & **max** of the `year` column, grouped by `director_name`.

In [37]:
data.groupby(["director_name"])["year"].min()

,year
director_name,
Adam McKay,2004
Adam Shankman,2001
Alejandro González Iñárritu,2000
Alex Proyas,1994
Alexander Payne,1999
...,...
Wes Craven,1984
Wolfgang Petersen,1981
Woody Allen,1977


In [42]:
data.groupby(["director_name"])["year"].max()

,year
director_name,
Adam McKay,2015
Adam Shankman,2012
Alejandro González Iñárritu,2015
Alex Proyas,2016
Alexander Payne,2013
...,...
Wes Craven,2011
Wolfgang Petersen,2006
Woody Allen,2013


In [ ]:
data.groupby(['director_name'])["year"].aggregate(['min', 'max'])

,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


**Note:** We can also use `.agg` instead of `.aggregate` (both are same)

In [44]:
data.groupby('year').agg({ 'vote_average': ['mean', 'min', 'max']})

vote_average          
             mean  min  max
year                       
1976     7.633333  7.2  8.0
1977     6.900000  6.1  7.8
1978     7.020000  5.9  7.9
1979     7.025000  5.6  8.0
1980     7.187500  6.4  7.9
1981     6.927273  6.0  7.9
1982     6.810000  5.5  7.9
1983     6.666667  4.5  8.0
1984     6.760000  6.1  7.3
1985     6.885714  5.9  8.0
1986     6.809091  5.8  7.8
1987     6.991667  6.1  7.6
1988     6.692308  5.7  7.6
1989     6.750000  5.5  7.6
1990     6.661111  5.8  8.2
1991     7.100000  6.6  8.1
1992     6.640909  5.7  8.0
1993     7.144444  6.1  8.3
1994     6.596774  5.0  8.3
1995     6.509091  4.5  8.1
1996     6.384783  5.4  7.8
1997     6.552174  4.1  7.9
1998     6.442222  4.9  7.9
1999     6.384375  5.1  8.3
2000     6.347458  4.4  8.1
2001     6.217742  3.7  8.0
2002     6.359420  3.4  8.0
2003     6.115517  4.4  8.1
2004     6.427869  4.7  7.7
2005     6.229577  3.1  7.7
2006     6.212329  3.6  8.0
2007     6.307407  3.2  7.9
2008     6.268657  3.0  8.2
2009     6.190909  3.9  7.9
2010     6.223810  4.2  8.1
2011     6.098507  4.1  7.7
2012     6.216327  4.9  7.8
2013     6.315789  3.3  7.9
2014     6.313208  4.1  8.1
2015     6.141463  3.9  7.7
2016     5.915385  4.5  7.1

---

### Group based Filtering

Group based filtering allows us to filter rows from each group by using conditional statements on each group rather than the whole dataframe.

**How to find the details of the movies by high budget directors?**

- Lets assume, high budget director -> any director with **atleast one movie with budget >100M**.

1. We can **group** the data by director and use `max` of the budget as aggregator.

In [45]:
# For each director we are printing the highest budget by that director
data_dir_budget = data.groupby("director_name")["budget"].max().reset_index()
data_dir_budget.head()

,director_name,budget
0,Adam McKay,100.0
1,Adam Shankman,80.0
2,Alejandro González Iñárritu,135.0
3,Alex Proyas,140.0
4,Alexander Payne,30.0


2. We can **filter** out the director names with **max budget >100M**.

In [47]:
# getting the directors which has highest budget which is >= 100million
names = data_dir_budget.loc[data_dir_budget["budget"] >= 100, "director_name"]

3. Finally, we can filter out the details of the movies by these directors.

In [48]:
data.loc[data['director_name'].isin(names)]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550.97
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661.00
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635.67
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834.94
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,48267,0.40,33,100.00,Mad Max,6.6,1213,1979,Apr,Thursday,George Miller,0,99.60
1451,48268,0.20,13,4.51,Swingers,6.8,253,1996,Oct,Friday,Doug Liman,0,4.31
1452,48274,0.00,5,2.61,Three,6.3,31,2010,Dec,Thursday,Tom Tykwer,0,2.61
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,0,3.16


**Can we filter groups in a single go using Lambda functions?** Yes!

In [ ]:
# only pick those groups whose group maximum budget >=100
data.groupby('director_name').filter(lambda x: x["budget"].max() >= 100)

# notice here we are filtering groups not rows

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0


Notice what's happening here?

- We first group data by director and then use `groupby().filter` function.
- **Groups are filtered if they do not satisfy the boolean criterion** specified by the function.
- This is called **Group Based Filtering**.

**Note:**
- We are filtering the **groups** here and **not the rows**.
- The result is **not a groupby object** but regular **Pandas DataFrame** with the **filtered groups eliminated**.

In [52]:
def check_risky(df):
  risky_flag = df["budget"] > df["revenue"].mean()
  df["risky"] = risky_flag
  return df

In [54]:
# Categorize a movie as risky or non-risky
# Here mu number is not static like above i.e 100
# Movie is risky, if its budget is greater than average revenue of all the movies of that director

# group by director_name
# now i need to apply a function on every group called check_risky
result = data.groupby("director_name").apply(check_risky);
result.head()

id_x  budget  popularity  revenue  \
director_name                                           
Adam McKay    176  43882   100.0          24   170.43   
              323  44151    72.5          12   162.97   
              366  44236    65.0          22   128.11   
              505  44503    50.0          38   173.65   
              839  45301    28.0          57   133.35   

                                                         title  vote_average  \
director_name                                                                  
Adam McKay    176                               The Other Guys           6.1   
              323  Talladega Nights: The Ballad of Ricky Bobby           6.2   
              366                                Step Brothers           6.5   
              505            Anchorman 2: The Legend Continues           6.0   
              839                                The Big Short           7.3   

                   vote_count  year month        day director_name  gender  \
director_name                                                                
Adam McKay    176        1383  2010   Aug     Friday    Adam McKay       0   
              323         491  2006   Aug     Friday    Adam McKay       0   
              366        1062  2008   Jul     Friday    Adam McKay       0   
              505         923  2013   Dec  Wednesday    Adam McKay       0   
              839        2607  2015   Dec     Friday    Adam McKay       0   

                   profit  risky  
director_name                     
Adam McKay    176   70.43  False  
              323   90.47  False  
              366   63.11  False  
              505  123.65  False  
              839  105.35  False

### Approach 2

1. For every director -> find the avg revenue
2. Merge this with the original dataframe
3. Row wise compare

In [58]:
data["revenue_mean"] = data.groupby("director_name")["revenue"].transform("mean")
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,revenue_mean
0,43597,237.0,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550.97,840.510000
1,43598,300.0,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661.00,453.858571
2,43599,245.0,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635.67,387.650000
3,43600,250.0,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834.94,528.435000
4,43602,258.0,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632.87,284.809091


---

### Group based Apply

- applying a function on grouped objects

**What if we want to do the transformation of a column using some column's agrregate**

Let's say, we want to filter the risky movies whose budget was even higher than the average revenue of the director from his other movies.

We can subtract the average `revenue` of a director from `budget` column, for each director.

In [ ]:
def func(x):
  # returns whether a movie is risky or not
  x["risky"] = x["budget"] - x["revenue"].mean() >= 0
  return x

data_risky = data.groupby("director_name", group_keys=False).apply(func)
data_risky

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,risky
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087,False
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000,False
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609,False
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099,False
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952,False
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130,False
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0,False
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0,False


**Note:**
- Setting `group_keys=True`, keeps the group key in the returned dataset.
- This will be default in future versions of Pandas.
- Keep it as False if want the normal behaviour.

**What did we do here?**

- Defined a custom function.
- Grouped data according to `director_name`.
- Subtracted the mean of `budget` from `revenue`.
- Used apply with the custom function on the grouped data.

Now let's see if there are any risky movies -

In [ ]:
data_risky.loc[data_risky["risky"]]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,risky
7,43608,200000000,107,586090727,Quantum of Solace,6.1,2965,2008,Oct,Thursday,Marc Forster,0,386090727,True
12,43614,380000000,135,1045713802,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,0,665713802,True
15,43618,200000000,37,310669540,Robin Hood,6.2,1398,2010,May,Wednesday,Ridley Scott,0,110669540,True
20,43624,209000000,64,303025485,Battleship,5.5,2114,2012,Apr,Wednesday,Peter Berg,0,94025485,True
24,43630,210000000,3,459359555,X-Men: The Last Stand,6.3,3525,2006,May,Wednesday,Brett Ratner,0,249359555,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,47224,5000000,7,3263585,The Sweet Hereafter,6.8,103,1997,May,Wednesday,Atom Egoyan,0,-1736415,True
1349,47229,5000000,3,4842699,90 Minutes in Heaven,5.4,40,2015,Sep,Friday,Michael Polish,0,-157301,True
1351,47233,5000000,6,0,Light Sleeper,5.7,15,1992,Aug,Friday,Paul Schrader,1,-5000000,True
1356,47263,15000000,10,0,Dying of the Light,4.5,118,2014,Dec,Thursday,Paul Schrader,1,-15000000,True


---